In [1]:
from env import *
set_seed(1234)

2024-01-25 14:34:35.797754: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-25 14:34:42.210403: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/nkmur/anaconda3/envs/web_con/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nkmur/anaconda3/envs/web_con/lib/python3.8/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
# import sys
# sys.path.insert(0,"/home/nkmur/lab/web_con/matsuo/Project/Dreamer/0109/dreamerv2")
# !pip uninstall -y dreamerv2
# !pip install -e ./dreamerv2/dreamerv2 -U
import dreamerv2
from dreamerv2.training.trainer import Trainer
from dreamerv2.training.evaluator import Evaluator

In [3]:
class Args:
    action_types = [
            # ActionTypes.NONE,
            ActionTypes.MOVE_COORDS,
            ActionTypes.CLICK_COORDS,
            # ActionTypes.DBLCLICK_COORDS,
            # ActionTypes.MOUSEDOWN_COORDS,
            # ActionTypes.MOUSEUP_COORDS,
            # ActionTypes.SCROLL_UP_COORDS,
            # ActionTypes.SCROLL_DOWN_COORDS,
        ] 
    action_types_num = len(action_types)
    screen_width, screen_height = 80, 80
    coord_bins = (51, 51)
    max_length = 500
    env_list = [
        "click-test-v1",
        "click-test-2-v1",
        "click-button-sequence-v1",
        "click-button-v1",
        # "click-link-v1",
        # "click-tab-v1",
        # "click-tab-2-easy"
        # "click-tab-v1-medium",
        # "click-tab-v1-hard",
        
        "click-option-v1",
        "click-checkboxes-v1",
        "click-widget-v1",
    ]
    step_limit = 10
    seed_fix = True
    device = "cuda"
    batch_size = 40
    seq_len = 5
    accumulated_loss = 4
    tokenizer_name = "microsoft/deberta-v3-base"
args = Args()

In [4]:
import argparse
import os
import torch
import numpy as np
import gym
from datetime import datetime
from dreamerv2.training.config import MiniWoBConfig
from dreamerv2.training.trainer import Trainer
from dreamerv2.training.evaluator import Evaluator
from torch.utils.tensorboard import SummaryWriter
logging.getLogger().setLevel(logging.ERROR)

env_list = args.env_list                                               

if torch.cuda.is_available() and args.device:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using :', device)  

vec_envs = VecEnvs(env_list, 
                action_types=args.action_types, coord_bins=args.coord_bins, max_length=args.max_length,
                screen_width=args.screen_width, screen_height=args.screen_height, 
                step_limit=args.step_limit, seed_fix=args.seed_fix,tokenizer_name=args.tokenizer_name)
obs_shape = np.array(vec_envs.vec_envs[0].observation_space.shape)[::-1]
act_shape = vec_envs.vec_envs[0].action_space.nvec
action_size = vec_envs.vec_envs[0].action_space.shape[0]

obs_dtype = np.float32
action_dtype = np.uint8
batch_size = args.batch_size
seq_len = args.seq_len

using : cuda


In [5]:

'''make dir for saving results'''
# chkpt_path = "/home/nkmur/lab/web_con/matsuo/Project/Dreamer/0113/results/0123_1534/"
chkpt_path = None
time = datetime.now().strftime("%m%d_%H%M")
if chkpt_path:
    result_dir = chkpt_path
else:
    result_dir = os.path.join('results', '{}'.format(time))
model_dir = os.path.join(result_dir, 'models')                                                  #dir to save learnt models
os.makedirs(model_dir, exist_ok=True)  
if args.tokenizer_name == "microsoft/deberta-v3-base":
    nl_embedding_size = 768
elif args.tokenizer_name == "microsoft/deberta-v3-xsmalll":
    nl_embedding_size = 384
    
config = MiniWoBConfig(
    env_list=env_list,
    obs_shape=obs_shape,
    act_shape=act_shape,
    action_size=action_size,
    max_length=args.max_length,
    obs_dtype = obs_dtype,
    action_dtype = action_dtype,
    seq_len = args.seq_len,
    batch_size = args.batch_size,
    model_dir=model_dir, 
    nl_embedding_size=nl_embedding_size,
    capacity=8*10**3,
    seed_steps=10**2,
    train_steps=2*10**4,
    # seed_steps=25,
    # train_steps = 20,
    train_every=10,
    eval_episode=1,
    save_every=10**3,
    # save_every=10,
    step_limit=args.step_limit,
    accumulated_loss = args.accumulated_loss,
    tokenizer_name=args.tokenizer_name,
)
if chkpt_path:
    # import pickle
    # with open(os.path.join(result_dir, 'config.pkl'), 'rb') as f:
    #     pickle.load(config, f)
    config_dict = config.__dict__
    trainer = Trainer(config, device, writer=None)
    evaluator = Evaluator(config, device, writer=None)
    restart_iter = trainer.restart_from_checkpoint(chkpt_path+"models/models_latest.pth")
    trainer.buffer.load(chkpt_path+"models/models_latest")
    # for i in range(len(env_list)):
    #     trainer.buffer.buffers[i].capacity = 4000
    #     trainer.buffer.buffers[i].idx = 0
    #     trainer.buffer.buffers[i].full = True
    #     trainer.buffer.buffers[i].observation = trainer.buffer.buffers[i].observation[:4000]
    #     trainer.buffer.buffers[i].action = trainer.buffer.buffers[i].action[:4000].astype(np.uint8)
    #     trainer.buffer.buffers[i].reward = trainer.buffer.buffers[i].reward[:4000].astype(np.int8)
    #     trainer.buffer.buffers[i].terminal = trainer.buffer.buffers[i].terminal[:4000]
    #     trainer.buffer.buffers[i].nl_obs_idx = trainer.buffer.buffers[i].nl_obs_idx[:4000].astype(np.uint8)
    #     gc.collect()
    writer = SummaryWriter(chkpt_path, purge_step=restart_iter)
    trainer.writer = writer
    evaluator.writer = writer
    print(f"Load from checkpoint {chkpt_path}, restart from {restart_iter}")
else:
    config_dict = config.__dict__
    with open(os.path.join(result_dir, 'config.txt'), 'w') as f:
        for k, v in config_dict.items():
            f.write('{}: {}\n'.format(k, v))
    
    restart_iter = 0
    writer = SummaryWriter(result_dir)   
    trainer = Trainer(config, device, writer)
    evaluator = Evaluator(config, device, writer)
    trainer.collect_seed_episodes(vec_envs)

Load from checkpoint /home/nkmur/lab/web_con/matsuo/Project/Dreamer/0113/results/0123_1534/, restart from 10000


In [6]:
config.nl_embedding_size

768

In [7]:
"""training loop"""
print('...training...')
train_metrics = {}
prev_rssmstate = trainer.RSSM._init_rssm_state(len(env_list))
prev_action = torch.zeros(len(env_list), trainer.action_size).to(trainer.device)
episode_actor_ent = [[] for _ in range(len(env_list))]
score = [0 for _ in range(len(env_list))]
scores = [[] for _ in range(len(env_list))]
best_mean_score = -np.inf
train_episodes = 0
best_save_path = os.path.join(model_dir, 'models_best.pth')

...training...


In [8]:
train_metrics['train_rewards'] = [0 for _ in range(len(env_list))]
train_metrics['action_ent'] = [[] for _ in range(len(env_list))]
train_metrics['train_steps'] = [[] for _ in range(len(env_list))]

In [9]:
obs = vec_envs.reset()
dones = torch.tensor([False for _ in range(len(env_list))]).to(trainer.device)
for iter in tqdm(range(restart_iter+1, trainer.config.train_steps), position=0, leave=True, desc='training_steps'):  
    gc.collect(),torch.cuda.empty_cache()
    if iter%trainer.config.train_every == 0:
        train_metrics = trainer.train_batch(train_metrics, iter)
        gc.collect(),torch.cuda.empty_cache()
    if iter%trainer.config.slow_target_update == 0:
        trainer.update_target()                
    if iter%trainer.config.save_every == 0:
        model_dir = config.model_dir
        save_path = os.path.join(model_dir, 'models_latest')
        trainer.save_model(iter)
        trainer.buffer.save(save_path)
        gc.collect(),torch.cuda.empty_cache()
    with torch.no_grad():
        embed = trainer.ObsEncoder(obs[0].to(trainer.device))
        _, posterior_rssm_state = trainer.RSSM.rssm_observe(embed, prev_action, 1-dones.unsqueeze(1)*1, prev_rssmstate)
        model_state = trainer.RSSM.get_model_state(posterior_rssm_state)
        action, action_dist = trainer.ActionModel(torch.cat([model_state, obs[1].to(trainer.device)], dim=-1))
        action = trainer.ActionModel.add_exploration(action, iter)
        action = torch.stack(action, dim=1).detach().cpu()
        a_d_list = []
        for a_d in action_dist:
            a_d_list.append(a_d.entropy().cpu())
        for i_env in range(len(env_list)):
            episode_actor_ent[i_env].append(torch.stack(a_d_list, dim=1).mean(dim=1)[i_env])
    next_obs, rew, done, _ = vec_envs.step(action)
    trainer.buffer.add((obs[0], obs[1]), action, rew, done)
    for i_env in range(len(env_list)):
        score[i_env] += rew[i_env]
        if done[i_env]:
            writer.add_scalar(f'train_rewards/{args.env_list[i_env]}', score[i_env], iter)
            writer.add_scalar(f'action_ent/{args.env_list[i_env]}', torch.tensor(episode_actor_ent[i_env]).mean(), iter)
            train_metrics['train_rewards'][i_env] = score[i_env]
            train_metrics['action_ent'][i_env] =  torch.tensor(episode_actor_ent[i_env]).mean().cpu()
            train_metrics['train_steps'][i_env] = iter
            scores[i_env].append(score[i_env])
            _s = vec_envs.vec_envs[i_env].reset(tokenizer=vec_envs.tokenizer, ocr=vec_envs.ocr, lm=vec_envs.lm, is_embed=True)
            obs[0][i_env], obs[1][i_env], dones[i_env], score[i_env] = _s[0], _s[1], False, 0
            prev_rssmstate.logit[i_env] = trainer.RSSM._init_rssm_state(1).logit
            prev_rssmstate.stoch[i_env] = trainer.RSSM._init_rssm_state(1).stoch
            prev_rssmstate.deter[i_env] = trainer.RSSM._init_rssm_state(1).deter
            prev_action[i_env] = torch.zeros(1, trainer.action_size).to(trainer.device)
            episode_actor_ent[i_env] = []
        else:
            obs[0][i_env], obs[1][i_env]= next_obs[0][i_env], next_obs[1][i_env]
            prev_rssmstate.logit[i_env] = posterior_rssm_state.logit[i_env]
            prev_rssmstate.stoch[i_env] = posterior_rssm_state.stoch[i_env]
            prev_rssmstate.deter[i_env] = posterior_rssm_state.deter[i_env]
            prev_action[i_env] = action[i_env]
    if sum([len(s) for s in scores])>100*len(env_list):
        [scores[i_env].pop(0) if len(scores[i_env])>100 else None for i_env in range(len(env_list))]
        current_average = np.mean(scores).mean()
        if current_average>best_mean_score:
            best_mean_score = current_average 
            print('saving best model with mean score : ', best_mean_score)
            save_dict = trainer.get_save_dict()
            torch.save(save_dict, best_save_path)
            del save_dict
            gc.collect(),torch.cuda.empty_cache()
    del embed, posterior_rssm_state, model_state, action, action_dist, a_d_list, next_obs, rew, done
    gc.collect(),torch.cuda.empty_cache()
'''evaluating probably best model'''
evaluator.eval_saved_agent(vec_envs, best_save_path)

training_steps:   2%|▏         | 207/9999 [15:46<9:17:27,  3.42s/it] 

saving best model with mean score :  0.08093194359288777


training_steps:  10%|█         | 1038/9999 [1:22:13<8:19:55,  3.35s/it] 

saving best model with mean score :  0.41


training_steps:  10%|█         | 1046/9999 [1:23:06<9:35:03,  3.85s/it] 

...

training_steps:  68%|██████▊   | 6796/9999 [9:08:39<3:34:36,  4.02s/it]

saving best model with mean score :  0.49714285714285716


training_steps: 100%|██████████| 9999/9999 [13:02:16<00:00,  4.69s/it]   


average evaluation score for model at /home/nkmur/lab/web_con/matsuo/Project/Dreamer/0113/results/0123_1534/models/models_best.pth = 0.5714285714285714


0.5714285714285714